# Analysis of cellphoneDB data

@mmm  
February 20, 2022

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# "scattermaps_mm.py" needs to be in the same folder as this notebook
import scattermaps_mm
from scattermaps_mm import scattermap

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
PATH = "/Users/mmm/BioPROJECTS/MAD1 & MVA/Results/220202 CellPhoneDB_ProbandAneu/"
DESKTOP = "/Users/mmm/Desktop/"

In [ ]:
# Functions

def order_ratios(ratios):
    """Order ratios per MAX values (descendent, both in rows and columns)"""
    
    ratios_ordered = ratios.copy()
    ratios_ordered["MAX"] = ratios_ordered.max(axis=1)
    ratios_ordered.loc["MAX_column", :] = ratio_L2_ordered.max(axis=0)
    ratios_ordered = ratios_ordered.sort_values(["MAX"], ascending=False, axis=0)
    ratios_ordered = ratios_ordered.drop("MAX", axis=1)
    ratios_ordered = ratios_ordered.sort_values(["MAX_column"], ascending=False, axis=1)
    ratios_ordered = ratios_ordered.drop("MAX_column", axis=0)
    return ratios_ordered

# Separate Plots Proband and Controls

In [ ]:
proband = pd.read_csv(DESKTOP + "Proband_significant_means_L1.txt", sep="\t")
proband = proband.set_index("interacting_pair")
proband.head(3)

In [ ]:
proband_pvals = pd.read_csv(DESKTOP + "Proband_pvalues.txt", sep="\t")
proband_pvals = proband_pvals.set_index("interacting_pair")
proband_pvals.head(3)

Select the first 40 interactions

In [ ]:
proband_ordered = pd.DataFrame(proband.index[:40])
proband_ordered = proband_ordered.set_index("interacting_pair")

In [ ]:
proband_pvals_ordered = proband_ordered.merge(proband_pvals, how="left", left_index=True, right_index=True)
proband_pvals_ordered = -np.log(proband_pvals_ordered)
proband_pvals_ordered = proband_pvals_ordered.replace(np.inf, 4)
proband_pvals_ordered.head(3)

In [ ]:
plt.figure(figsize=(20,10))
ax = scattermap(proband.iloc[:40,:], marker_size=proband_pvals_ordered, cmap='Reds', factor=60, square=True) #marker_size=pre_res_fdr
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

### Controls

In [ ]:
controls = pd.read_csv(DESKTOP + "Controls_significant_means.txt", sep="\t")
controls = controls.set_index("interacting_pair")
controls.head(3)

In [ ]:
controls_pvals = pd.read_csv(DESKTOP + "Proband_pvalues.txt", sep="\t")
controls_pvals = controls_pvals.set_index("interacting_pair")
controls_pvals.head(3)

In [ ]:
proband_ordered = pd.DataFrame(proband.index[:40])
proband_ordered = proband_ordered.set_index("interacting_pair")

In [ ]:
controls_ordered = proband_ordered.merge(controls, how="left", left_index=True, right_index=True)
controls_pvals_ordered = proband_ordered.merge(controls_pvals, how="left", left_index=True, right_index=True)
controls_pvals_ordered = -np.log(controls_pvals_ordered)
controls_pvals_ordered = controls_pvals_ordered.replace(np.inf, 4)
controls_pvals_ordered.head(3)

In [ ]:
plt.figure(figsize=(20,10))
ax = scattermap(controls_ordered, marker_size=controls_pvals_ordered, cmap='Reds', factor=60, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

# Ratio between proband and controls (L1)

In [ ]:
proband = pd.read_csv(DESKTOP + "Proband_significant_means_L1.txt", sep="\t")
proband = proband.set_index("interacting_pair")
print(proband.shape)
proband.head(3)

In [ ]:
controls = pd.read_csv(DESKTOP + "Controls_significant_means_L1.txt", sep="\t")
controls = controls.set_index("interacting_pair")
print(controls.shape)
controls.head(3)

In [ ]:
# Order controls as in proband
proband_order = proband[[]].copy()
controls_ordered = proband_order.merge(controls, how="left", left_index=True, right_index=True)

In [ ]:
# Ratio
ratio = proband / controls_ordered
ratio.head(3)

In [ ]:
# Calculate and order MAX values
ratio_sorted = order_ratios(ratio)

In [ ]:
plt.figure(figsize=(20,10))
ax = scattermap(ratio_sorted.iloc[:20,:], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

### Analyze L1 interactions in L2

In [ ]:
## MUlktiple

interactions = ["ESAM_ESAM", "THBS1_a2Bb3 complex", "PF4_CXCR3", "CLEC2B_KLRF1", 
                "ICAM1_AREG", "ICAM1_aLb2 complex", "ICAM1_ITGAL", "ICAM4_aLb2 complex", "ICAM1_SPN",
                "CCL5_CCR1", "PECAM1_CD38", 
                "TNFSF14_TNFRSF14", "CD160_TNFRSF14", 
                "TGFB1_TGFBR3", "TGFB1_TGFbeta receptor1", 
                "ALOX5_ALOX5AP", "CCL4_SLC7A1", "ICOSLG_ICOS",
                "IL15 receptor_IL15"
               ]

#"TNFSF14_LTBR", "TNFSF4_TNFRSF4",

In [ ]:
ratio_L2 = pd.read_csv(DESKTOP + "ratio_L2.tsv", sep="\t")
ratio_L2 = ratio_L2.set_index("interacting_pair")

In [ ]:
ratio_L2_selected = ratio_L2[ratio_L2.index.isin(interactions)]
ratio_L2_ordered = order_ratios(ratio_L2_selected)
ratio_L2_ordered.to_csv(DESKTOP + "ratio_L2_selected.tsv", sep="\t")

In [ ]:
plt.figure(figsize=(30,6))
ax = scattermap(ratio_L2_ordered.iloc[:,:100], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

## Separate per families

In [ ]:
# Platelets
i = ["PECAM1_CD38","ESAM_ESAM", "PF4_CXCR3"]
ratio_L2_selected = ratio_L2[ratio_L2.index.isin(i)]
ratio_L2_selected = order_ratios(ratio_L2_selected)

In [ ]:
plt.figure(figsize=(7,4))
ax = scattermap(ratio_L2_selected.iloc[:,:15], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=4, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
# NK-gdT
i = ["TNFSF14_TNFRSF14", "CD160_TNFRSF14", "CCL5_CCR1", "IL15 receptor_IL15"]
ratio_L2_selected = ratio_L2[ratio_L2.index.isin(i)]
ratio_L2_selected = order_ratios(ratio_L2_selected)

In [ ]:
plt.figure(figsize=(10,4))
ax = scattermap(ratio_L2_selected.iloc[:,:20], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=4, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
# CD8
i = ["CLEC2B_KLRF1", "CCL4_SLC7A1"]
ratio_L2_selected = ratio_L2[ratio_L2.index.isin(i)]
ratio_L2_selected = order_ratios(ratio_L2_selected)

In [ ]:
plt.figure(figsize=(10,4))
ax = scattermap(ratio_L2_selected.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2.5, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
# dendritic
i = ["ICOSLG_ICOS","THBS1_a2Bb3 complex", "ICAM1_aLb4 complex", "ALOX5_ALOX5AP", 
     "TGFB1_TGFBR3", "TGFB1_TGFbeta receptor1"]
ratio_L2_selected = ratio_L2[ratio_L2.index.isin(i)]
ratio_L2_selected = order_ratios(ratio_L2_selected)

In [ ]:
plt.figure(figsize=(10,4))
ax = scattermap(ratio_L2_selected.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2.2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
# B
i = ["ICAM1_AREG", "ICAM1_aLb2 complex", "ICAM1_ITGAL", "ICAM1_SPN",]
ratio_L2_selected = ratio_L2[ratio_L2.index.isin(i)]
ratio_L2_selected = order_ratios(ratio_L2_selected)

In [ ]:
plt.figure(figsize=(10,4))
ax = scattermap(ratio_L2_selected.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2.2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

### Analyze L1 interactions in L3

In [ ]:
ratio_L3 = pd.read_csv(DESKTOP + "ratio_L3.tsv", sep="\t")
ratio_L3 = ratio_L3.set_index("interacting_pair")

In [ ]:
ratio_L3_selected = ratio_L3[ratio_L3.index.isin(interactions)]
ratio_L3_selected = order_ratios(ratio_L3_selected)
ratio_L3_selected.to_csv(DESKTOP + "ratio_L3_selected.tsv", sep="\t")

In [ ]:
plt.figure(figsize=(20,6))
ax = scattermap(ratio_L3_selected.iloc[:,:50], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

## Separate per families

In [ ]:
# Platelets
i = ["ESAM_ESAM", "PF4_CXCR3"]
ratio_L3_selected = ratio_L3[ratio_L3.index.isin(i)]
ratio_L3_selected = order_ratios(ratio_L3_selected)

In [ ]:
plt.figure(figsize=(7,3))
ax = scattermap(ratio_L3_selected.iloc[:,:10], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=4, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
# NKs_gdT
i = ["TNFSF14_TNFRSF14", "CD160_TNFRSF14", "CLEC2B_KLRF1", "IL15 receptor_IL15"]
ratio_L3_selected = ratio_L3[ratio_L3.index.isin(i)]
ratio_L3_selected = order_ratios(ratio_L3_selected)

In [ ]:
plt.figure(figsize=(14,6))
ax = scattermap(ratio_L3_selected.iloc[:,:50], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=1.6, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
# dendritic
i = ["ICAM1_AREG", "ICAM1_aLb2 complex", "ICAM1_ITGAL", "ICAM4_aLb2 complex", "ICAM1_SPN",
     "TNFSF14_TNFRSF14", "PECAM1_CD38", "ALOX5_ALOX5AP",]
ratio_L3_selected = ratio_L3[ratio_L3.index.isin(i)]
ratio_L3_selected = order_ratios(ratio_L3_selected)

In [ ]:
plt.figure(figsize=(14,6))
ax = scattermap(ratio_L3_selected.iloc[:,:50], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=1.6, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
# CD8
i = ["THBS1_a2Bb3 complex", "CLEC2B_KLRF1", "CCL5_CCR1", "TGFB1_TGFBR3", "TGFB1_TGFbeta receptor1", "ICOSLG_ICOS", ]
ratio_L3_selected = ratio_L3[ratio_L3.index.isin(i)]
ratio_L3_selected = order_ratios(ratio_L3_selected)

In [ ]:
plt.figure(figsize=(14,6))
ax = scattermap(ratio_L3_selected.iloc[:,:50], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=1.6, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

In [ ]:
## OLDER

In [ ]:
# Import data ordered by MAX value , only columns >1.3
ratio = pd.read_csv(DESKTOP + "ratio_MAX_ordered_clean_1.3.tsv", sep="\t")
ratio = ratio.set_index("interacting_pair")
ratio = ratio.drop("MAX", axis=1)
print(ratio.shape)
ratio.head(3)

In [ ]:
plt.figure(figsize=(10,6))
ax = scattermap(ratio, cmap='coolwarm', vmin=0.6, vmax=1.8, factor=1.1, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

# Ratio between proband and controls (L2)

In [ ]:
proband = pd.read_csv(DESKTOP + "Proband_significant_means_L2.txt", sep="\t")
proband = proband.set_index("interacting_pair")
print(proband.shape)

controls = pd.read_csv(DESKTOP + "Controls_significant_means_L2.txt", sep="\t")
controls = controls.set_index("interacting_pair")
print(controls.shape)

In [ ]:
# Order controls as in proband
proband_order = proband[[]].copy()
controls_ordered = proband_order.merge(controls, how="left", left_index=True, right_index=True)

In [ ]:
# Use only same columns
column_order = proband.columns
controls_ordered = controls_ordered[column_order]
controls_ordered

In [ ]:
ratios_L2 = proband / controls_ordered
ratios_L2.head(3)

In [ ]:
ratios_L2.to_csv(DESKTOP + "ratio_L2.tsv", sep="\t")

In [ ]:
# Order by MAX values
ratio_L2_ordered = ratios_L2.copy()
ratio_L2_ordered = order_ratios(ratio_L2_ordered)
ratio_L2_ordered.to_csv(DESKTOP + "ratio_L2_ordered.tsv", sep="\t")

In [ ]:
plt.figure(figsize=(20,10))
ax = scattermap(ratio_L2_ordered.iloc[:30,:80], cmap='coolwarm', vmin=0, vmax=2, factor=1.1, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
# Select specific interactions
i = ["JAM3_JAM3", "CD28_CD86", "SELP_SELPLG", "NOTCH1_JAG1", "LGALS9_HAVCR2", "SELP_CD24", "JAG1_NOTCH2",
     "CLEC2B_KLRF1", "ICAM1_AREG", "ICAM1_alb2 complex", "ICAM1_ITGAL", "CXCR3_CCL20", "ICOSLG_ICOS",
     "CCR6_CCL20", "CD46_JAG1", "EPHA4_EFNB1", "SELP_CD34", "ADGRG5_FAM3C", "ICAM1_SPN", "TNFSF9_ADGRG5",
     "JAM3_aMb2 complex", "TNF_ICOS", "COPA_P2RY6", "LAIR1_LILRB4", "EPHB1_EFNB1", "ICAM_aLb2 complex",
     "NCR3_BAG6"
]

i_platelet = ["JAM3_JAM3", "SELP_SELPLG", "SELP_CD24", "SELP_CD34", ]
i_ILC_HSPC = ["NOTCH1_JAG1", "LGALS9_HAVCR2", "JAG1_NOTCH2", "CD46_JAG1", "EPHA4_EFNB1", "TNFSF9_ADGRG5",]
i_Tcells = ["CLEC2B_KLRF1", "CD28_CD86", "CXCR3_CCL20", "ICOSLG_ICOS", "CCR6_CCL20", ]
i_Bcells = ["ICAM1_AREG", "ICAM1_SPN", ]


### Select specific cell types

In [ ]:
platelet_columns = [x for x in ratios_L2 if x.startswith("Plate")]
NK_gdT_columns = [x for x in ratios_L2 if (x.startswith("NK") or x.startswith("gdT"))]
dnT_columns = [x for x in ratios_L2 if (x.startswith("dnT") or x.startswith("Treg"))]
CD4_CD8_columns = [x for x in ratios_L2 if (x.startswith("CD4") or x.startswith("CD8"))]
ILC_HSPC_columns = [x for x in ratios_L2 if (x.startswith("ILC") or x.startswith("HSPC"))]
B_columns = [x for x in ratios_L2 if x.startswith("B")]
dendritic_columns = [x for x in ratios_L2 if (x.startswith("pDC") or x.startswith("cDC2") or x.startswith("CD14") or x.startswith("CD16"))]

In [ ]:
celltype = ratio_L2[dnT_columns]
celltype = order_ratios(celltype)

plt.figure(figsize=(14,6))
ax = scattermap(celltype.iloc[:25,:20], cmap='coolwarm', vmin=0, vmax=2, factor=1.1, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png", dpi=300)

## Ratio between proband and controls (L3)

In [ ]:
proband = pd.read_csv(DESKTOP + "Proband_significant_means_L3.txt", sep="\t")
proband = proband.set_index("interacting_pair")
print(proband.shape)
proband.head(3)

In [ ]:
controls = pd.read_csv(DESKTOP + "Controls_significant_means_L3.txt", sep="\t")
controls = controls.set_index("interacting_pair")
print(controls.shape)
controls.head(3)

In [ ]:
# Order controls as in proband

In [ ]:
proband_order = proband[[]].copy()

In [ ]:
controls_ordered = proband_order.merge(controls, how="left", left_index=True, right_index=True)

In [ ]:
controls_ordered

In [ ]:
# Use only same columns
column_order = proband.columns
controls_ordered = controls_ordered[column_order]
controls_ordered

In [ ]:
ratio = proband / controls_ordered
ratio.head(3)

In [ ]:
ratio.to_csv(DESKTOP + "ratio_L3.tsv", sep="\t")

In [ ]:
# Import data ordered by MAX value
ratio = pd.read_csv(DESKTOP + "ratio_L3_clean_1.3.tsv", sep="\t")
ratio = ratio.set_index("interacting_pair")
ratio = ratio.drop("MAX", axis=1)
print(ratio.shape)
ratio.head(3)

In [ ]:
plt.figure(figsize=(20,30))
ax = scattermap(ratio.iloc[:,:50], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
## Filter specific interactions
# Import data ordered by MAX value
ratio = pd.read_csv(DESKTOP + "ratio_L3.tsv", sep="\t")
ratio = ratio.set_index("interacting_pair")
print(ratio.shape)
ratio.head(3)

In [ ]:
ratio_CCL5 = ratio[ratio.index.str.startswith("CCL5")]
ratio_CCL5_sorted = ratio_CCL5.sort_values(["CCL5_CCR1", "CCL5_CCR4"], ascending=False, axis=1)
ratio_CCL5_sorted

In [ ]:
plt.figure(figsize=(10,5))
ax = scattermap(ratio_CCL5_sorted.iloc[:,:10], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_PF4 = ratio[ratio.index.str.startswith("PF4")]
ratio_PF4

In [ ]:
ratio_PF4_sorted = ratio_PF4.sort_values(["PF4_CXCR3"], ascending=False, axis=1)
ratio_PF4_sorted

In [ ]:
plt.figure(figsize=(10,5))
ax = scattermap(ratio_PF4_sorted.iloc[:,:20], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_ESAM = ratio[ratio.index.str.startswith("ESAM")]
ratio_ESAM

In [ ]:
ratio_ESAM_sorted = ratio_ESAM.sort_values(["ESAM_ESAM"], ascending=False, axis=1)
ratio_ESAM_sorted

In [ ]:
plt.figure(figsize=(10,5))
ax = scattermap(ratio_ESAM_sorted.iloc[:,:20], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_THBS1 = ratio[ratio.index.str.startswith("THBS1")]
ratio_THBS1

In [ ]:
ratio_THBS1_sorted = ratio_THBS1.sort_values(["THBS1_a2Bb3 complex", "THBS1_a3b1 complex", 
                                            "THBS1_aVb3 complex"], ascending=False, axis=1)
ratio_THBS1_sorted

In [ ]:
plt.figure(figsize=(10,5))
ax = scattermap(ratio_THBS1_sorted.iloc[:,:20], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_CLEC2 = ratio[ratio.index.str.startswith("CLEC2")]
ratio_CLEC2

In [ ]:
ratio_CLEC2_sorted = ratio_CLEC2.sort_values(["CLEC2B_KLRF1"], ascending=False, axis=1)
ratio_CLEC2_sorted

In [ ]:
plt.figure(figsize=(10,5))
ax = scattermap(ratio_CLEC2_sorted.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_ICAM = ratio[ratio.index.str.startswith("ICAM")]
ratio_ICAM

In [ ]:
ratio_ICAM_sorted = ratio_ICAM.sort_values(["ICAM1_ITGAL", "ICAM1_AREG", ], ascending=False, axis=1)
ratio_ICAM_sorted

In [ ]:
plt.figure(figsize=(20,6))
ax = scattermap(ratio_ICAM_sorted.iloc[:,:60], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_CCL5 = ratio[ratio.index.str.startswith("CCL5")]
ratio_CCL5

In [ ]:
ratio_CCL5_sorted = ratio_CCL5.sort_values(["CCL5_CCR1", ], ascending=False, axis=1)
ratio_CCL5_sorted

In [ ]:
plt.figure(figsize=(14,3))
ax = scattermap(ratio_CCL5_sorted.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_PECAM = ratio[ratio.index.str.startswith("PECAM")]
ratio_PECAM

In [ ]:
ratio_PECAM_sorted = ratio_PECAM.sort_values(["PECAM1_CD38"], ascending=False, axis=1)
ratio_PECAM_sorted

In [ ]:
plt.figure(figsize=(14,3))
ax = scattermap(ratio_PECAM_sorted.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_TNFSF14 = ratio[ratio.index.str.startswith("TNFSF14")]
ratio_TNFSF14

In [ ]:
ratio_TNFSF14_sorted = ratio_TNFSF14.sort_values(["TNFSF14_LTBR", "TNFSF14_TNFRSF14"], ascending=False, axis=1)
ratio_TNFSF14_sorted

In [ ]:
plt.figure(figsize=(14,3))
ax = scattermap(ratio_TNFSF14_sorted.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_TGFB1 = ratio[ratio.index.str.startswith("TGFB1")]
ratio_TGFB1

In [ ]:
ratio_TGFB1_sorted = ratio_TGFB1.sort_values(["TGFB1_TGFbeta receptor1", "TGFB1_TGFBR3", ], ascending=False, axis=1)
ratio_TGFB1_sorted

In [ ]:
plt.figure(figsize=(14,3))
ax = scattermap(ratio_TGFB1_sorted.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_CD160 = ratio[ratio.index.str.startswith("CD160")]
ratio_CD160

In [ ]:
ratio_CD160_sorted = ratio_CD160.sort_values(["CD160_TNFRSF14", ], ascending=False, axis=1)
ratio_CD160_sorted

In [ ]:
plt.figure(figsize=(14,3))
ax = scattermap(ratio_CD160_sorted.iloc[:,:40], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_CCL4 = ratio[ratio.index.str.startswith("CCL4")]
ratio_CCL4

In [ ]:
ratio_CCL4_sorted = ratio_CCL4.sort_values(["CCL4_SLC7A1", ], ascending=False, axis=1)
ratio_CCL4_sorted

In [ ]:
plt.figure(figsize=(14,3))
ax = scattermap(ratio_CCL4_sorted.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_IFNG = ratio[ratio.index.str.startswith("IFNG")]
ratio_IFNG

In [ ]:
ratio_IFNG_sorted = ratio_IFNG.sort_values(["IFNG_Type II IFNR", ], ascending=False, axis=1)
ratio_IFNG_sorted

In [ ]:
plt.figure(figsize=(14,3))
ax = scattermap(ratio_IFNG_sorted.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
## MUlktiple

interactions = ["ESAM_ESAM", "THBS1_a2Bb3 complex", "PF4_CXCR3", "CLEC2B_KLRF1", 
               "ICAM4_aLb2 complex", "ICAM1_ITGAL", "ICAM1_aLb2 complex", "ICAM1_AREG",
               "CCL5_CCR1", "PECAM1_CD38", "TNFSF14_LTBR", "TNFSF14_TNFRSF14",
               "TGFB1_TGFbeta receptor1", "TGFB1_TGFBR3", "CD160_TNFRSF14", "CCL4_SLC7A1",
               "IFNG_Type II IFNR",
               ]

ratio_selected = ratio[ratio.index.isin(interactions)]
ratio_selected

In [ ]:
ratio_selected.loc["Total", :] = ratio_selected.sum(axis=0)
ratio_selected_sorted = ratio_selected.sort_values(["Total", ], ascending=False, axis=1)
ratio_selected_sorted

In [ ]:
ratio_selected_sorted_noTotal = ratio_selected_sorted.drop("Total", axis=0)
ratio_selected_sorted_noTotal

In [ ]:
plt.figure(figsize=(12,6))
ax = scattermap(ratio_selected_sorted_noTotal.iloc[:,:40], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
ratio_selected_sorted.columns

In [ ]:
NK_columns = [x for x in ratio_selected_sorted.columns if x.startswith("NK")]
len(NK_columns)

In [ ]:
ratio_selected_sorted_NK = ratio_selected_sorted[NK_columns]

In [ ]:
ratio_selected_sorted_NK = ratio_selected_sorted_NK.sort_values(["Total", ], ascending=False, axis=1)
ratio_selected_sorted_NK_noTotal = ratio_selected_sorted_NK.drop("Total", axis=0)

In [ ]:
plt.figure(figsize=(18,8))
ax = scattermap(ratio_selected_sorted_NK_noTotal.iloc[:,:60], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
gdT_columns = [x for x in ratio_selected_sorted.columns if x.startswith("gdT")]
len(gdT_columns)

In [ ]:
ratio_selected_sorted_gdT = ratio_selected_sorted[gdT_columns]
ratio_selected_sorted_gdT = ratio_selected_sorted_gdT.sort_values(["Total", ], ascending=False, axis=1)
ratio_selected_sorted_gdT_noTotal = ratio_selected_sorted_gdT.drop("Total", axis=0)

In [ ]:
plt.figure(figsize=(18,8))
ax = scattermap(ratio_selected_sorted_gdT_noTotal.iloc[:,:60], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

In [ ]:
platelet_columns = [x for x in ratio_selected_sorted.columns if x.startswith("Platelet")]
len(platelet_columns)

In [ ]:
ratio_selected_sorted_platelet = ratio_selected_sorted[platelet_columns]
ratio_selected_sorted_platelet = ratio_selected_sorted_platelet.sort_values(["Total", ], ascending=False, axis=1)
ratio_selected_sorted_platelet_noTotal = ratio_selected_sorted_platelet.drop("Total", axis=0)

In [ ]:
plt.figure(figsize=(20,6))
ax = scattermap(ratio_selected_sorted_platelet_noTotal.iloc[:,:30], cmap='coolwarm', vmin=0.5, vmax=1.5, factor=2, square=True) 
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")

## Separate ligand and receptors

February 2, 2022

In [ ]:
codes = pd.read_csv(PATH + "L1_codes.tsv", sep="\t")
codes_dic = dict(zip(codes["interacting_pair"], codes["celltype"]))

data = pd.read_csv(PATH + "Proband_Aneu_vs_EU_L1.tsv", sep="\t")

In [ ]:
data_A = data[(data["Select Aneu"] == "B") |  (data["Select Aneu"] == "A or B")]
data_B = data[(data["Select Aneu"] == "A") |  (data["Select Aneu"] == "A or B")]

In [ ]:
data_A = data_A.set_index("interacting_pair")
data_B = data_B.set_index("interacting_pair")
data_A

In [ ]:
codes_A = codes[codes["Aneu (A - B - AB - O)"] == "A"]
codes_B = codes[codes["Aneu (A - B - AB - O)"] == "B"]

In [ ]:
columns_A = codes_A["interacting_pair"].tolist()
columns_B = codes_B["interacting_pair"].tolist()
print(len(columns_A))
print(len(columns_B))

In [ ]:
data_A = data_A[columns_A]
data_B = data_B[columns_B]
data_A.shape

In [ ]:
data_A = data_A.rename(columns=codes_dic)
data_B = data_B.rename(columns=codes_dic)
data_A.to_csv(DESKTOP + "data_A.tsv", sep="\t")
data_B.to_csv(DESKTOP + "data_B.tsv", sep="\t")

In [ ]:
data_AB = pd.read_csv(DESKTOP + "data_AB2.tsv", sep="\t")

In [ ]:
data_AB = data_AB.set_index("interacting_pair")

In [ ]:
plt.figure(figsize=(20,10))
ax = scattermap(data_AB, cmap='Reds', factor=2, square=True) #marker_size=pre_res_fdr
plt.tight_layout()
plt.savefig(DESKTOP + "dotPlot.png")